In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Processing results
import json

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
from sklearn.linear_model import LinearRegression

# Processing
from sklearn.preprocessing import StandardScaler

# Data imputation
from sklearn.impute import SimpleImputer

# Pipeline
from sklearn.pipeline import Pipeline

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score


# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

<IPython.core.display.Javascript object>

# Functions and definitions

In [3]:
index_to_save = 11

<IPython.core.display.Javascript object>

In [4]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [5]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "partner_i",
    "Features": "Chemical + Mineralogical + Feature Engineering",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "Linear Regression",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [6]:
df = pd.read_csv("../../../../../../data/processed/partner_i-Oficial/cement-shipping.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we keep only chemical and mineralogical features yielded by the same testing method/procedure

In [7]:
df_copy = df.drop(
    [
        # Properties
        "Blaine",
        "Initial setting time",
        "Final setting time",
        "Density",
        "#200",
        "#325",
        "CS3",
        "CS7",
        
        # Removing One-Hot encoding variables
        "Cement_Type",
        
        
        # Chemical Composition
        # Reason: Loss on Ignition is the only feature
        # that belongs to chemical composition in which was 
        # measured by a different method, namely manual
        
    ],
    axis=1,
).copy()

<IPython.core.display.Javascript object>

## Feature Engineering

In [8]:
# Feature Engineering over Chemical Features
ch_features = ["CaO", "MgO", "Na2O", "Al2O3", "SiO2", "SO3", "K2O", "Fe2O3"]

df_copy["std_ch_feats"] = df_copy[ch_features].std(ddof=0, axis=1)

df_copy["ratio_CaO_to_SiO2"] = df_copy["CaO"] / df_copy["SiO2"]
df_copy["ratio_MgO_to_CaO"] = df_copy["MgO"] / df_copy["CaO"]

# Feature Engineering over Mineralogical Features
mi_features_set1 = [
    "Alpha C2S",
    "Beta C2S",
    "Gamma C2S",
    "C3A",
    "Cubic C3A",
    "Orthorhombic C3A",
]

mi_features_set2 = [
    "Portlandite",
    "Periclase",
    "Arcanite",
    "Aphthitalite",
    "Gypsum",
    "Bassanite",
    "Anhydrite",
    "Calcite",
    "Dolomite",
    "Quartz",
]
df_copy["mean_mi_set1_feats"] = df_copy[mi_features_set1].mean(axis=1)
df_copy["mean_mi_set2_feats"] = df_copy[mi_features_set2].mean(axis=1)

df_copy["ratio_Aluminate_to_Ferrite"] = df_copy["C3A"] / df_copy["C4AF"]

<IPython.core.display.Javascript object>

# 1. Linear Regression

<h2>1. Dataset: df_no_cs</h2> <br>In this dataset the CS1, CS3  and CS7 variables are not considered. Only Chemical and mineralogical features measured by the same method. For this particular dataset, all chemical features, with the exception of LOI were measured by XRF and XRD methods.

In [9]:
y = df_copy.pop("CS28").values
x = df_copy.drop(["Date"], axis=1)
dates = df["Date"].copy()

<IPython.core.display.Javascript object>

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 123 samples each
    2. 90% train (1111 samples each fold)
    3. 10% test (123 samples each fold)
<b>Total:</b> 100 models<br>

In [10]:
repeats = 3
n_splits = 5
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Repeated KFold"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Repeated Cross Validation:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -3.324 (0.040)
MAE: -2.542 (0.033)
MAPE: -0.061 (0.001)
R2: 0.566 (0.011)


******
[TEST]
******
RMSE: -3.443 (0.161)
MAE: -2.631 (0.138)
MAPE: -0.063 (0.003)
R2: 0.531 (0.052)




<IPython.core.display.Javascript object>

In [11]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                       
0  Chemical + Mineralogical + Feature Engineering  Linear Regression   

  Cross Validation RMSE Test            MAE Test           MAPE Test           \
                        mean       std      mean       std      mean      std   
0   Repeated KFold  3.442696  0.160652  2.631255  0.138236  0.063168  0.00316   

    R2 Test            
       mean       std  
0  0.530535  0.051513

<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 246 samples
    2. 50% train (123 samples each fold)
    3. 50% test (123 samples each fold)
<b>Total:</b> 5 models<br>

In [12]:
n_splits = 5
train_size = 0.8

pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
results_dict_copy[
    "Cross Validation Params"
] = '{"N_Splits": 5, "Repeats": 1, "train_size": 0.8}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -2.863 (0.330)
MAE: -2.193 (0.321)
MAPE: -0.052 (0.008)
R2: 0.679 (0.049)


******
[TEST]
******
RMSE: -3.753 (0.446)
MAE: -2.931 (0.408)
MAPE: -0.070 (0.010)
R2: 0.412 (0.118)




<IPython.core.display.Javascript object>

In [13]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                       
0  Chemical + Mineralogical + Feature Engineering  Linear Regression   
1  Chemical + Mineralogical + Feature Engineering  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  3.753366  0.445944  2.930611  0.408041   
1              Repeated KFold  3.442696  0.160652  2.631255  0.138236   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.069973  0.010322  0.412466  0.117655  
1  0.063168  0.003160  0.530535  0.051513

<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 114, 226, 338, 450, 562, 675, 787, 899, 1011, 1123 samples each fold
    2. Test: 112 samples each fold
<b>Total:</b> 10 models<br>

In [14]:
n_splits = 5
gap = 0
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = TimeSeriesSplit(gap=gap, max_train_size=None, n_splits=n_splits, test_size=None)

scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Time Series Split"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 1, "Gap": 0}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -3.122 (0.261)
MAE: -2.433 (0.190)
MAPE: -0.058 (0.005)
R2: 0.599 (0.054)


******
[TEST]
******
RMSE: -4.035 (0.581)
MAE: -3.129 (0.433)
MAPE: -0.074 (0.010)
R2: 0.357 (0.150)




<IPython.core.display.Javascript object>

In [15]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                       
0  Chemical + Mineralogical + Feature Engineering  Linear Regression   
1  Chemical + Mineralogical + Feature Engineering  Linear Regression   
2  Chemical + Mineralogical + Feature Engineering  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  3.753366  0.445944  2.930611  0.408041   
1              Repeated KFold  3.442696  0.160652  2.631255  0.138236   
2           Time Series Split  4.035213  0.581476  3.129473  0.432922   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.069973  0.010322  0.412466  0.117655  
1  0.063168  0.003160  0.530535  0.051513  
2  0.074318  0.009953  0.356876  0.150387

<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 987
    2. Test: 247
<b>Total:</b> 1 model<br>

In [16]:
test_size = 0.2

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=test_size, random_state=SEED, shuffle=False
)
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)

pipeline.fit(x_train, y_train)

y_train_pred = pipeline.predict(x_train)
y_test_pred = pipeline.predict(x_test)

scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Out of time Split"
results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(
    results_dict_copy, {key: [value] for key, value in scores.items()}
)
results_to_save.append(df_results)

******
[TRAIN]
******
RMSE: 3.410 (0.000)
MAE: 2.637 (0.000)
MAPE: 0.063 (0.000)
R2: 0.545 (0.000)


******
[TEST]
******
RMSE: 3.081 (0.000)
MAE: 2.396 (0.000)
MAPE: 0.056 (0.000)
R2: 0.616 (0.000)




<IPython.core.display.Javascript object>

In [17]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                       
0  Chemical + Mineralogical + Feature Engineering  Linear Regression   
1  Chemical + Mineralogical + Feature Engineering  Linear Regression   
2  Chemical + Mineralogical + Feature Engineering  Linear Regression   
3  Chemical + Mineralogical + Feature Engineering  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  3.753366  0.445944  2.930611  0.408041   
1           Out of time Split  3.081432  0.000000  2.396342  0.000000   
2              Repeated KFold  3.442696  0.160652  2.631255  0.138236   
3           Time Series Split  4.035213  0.581476  3.129473  0.432922   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.069973  0.010322  0.412466  0.117655  
1  0.055980  0.000000  0.616404  0.000000  
2  0.063168  0.003160  0.530535  0.051513  
3  0.074318  0.009953  0.356876  0.150387

<IPython.core.display.Javascript object>

In [18]:
pipeline.named_steps["estimator"].coef_

array([ 1.69106902e+02,  1.48648081e+01, -1.32552960e+00, -3.77058892e+00,
        3.42059908e+01, -9.34899931e+00, -2.74337093e+00, -3.33096018e+00,
        2.89731520e-01, -2.01964577e-01, -1.67269941e+00,  1.85123762e+01,
        4.46743369e+01,  1.09444784e+01,  1.03453948e+00,  1.44123916e+01,
        1.49217301e+01,  1.05062455e+01,  9.90991525e-02,  8.18842359e-02,
        7.35621083e-02,  2.67011925e-01,  5.61039262e-01,  1.87309948e-01,
        2.58517962e-01, -1.78676021e-02,  1.72942066e+00,  1.87635346e-01,
        1.90973746e-01, -1.81129612e+02,  1.39034099e+01, -2.68427485e+01,
       -6.96367347e+01, -3.79146462e+00,  2.66758520e+00])

<IPython.core.display.Javascript object>

In [19]:
pipeline.named_steps["estimator"].intercept_

43.39817346456112

<IPython.core.display.Javascript object>

In [20]:
coeff = pipeline.named_steps["estimator"].coef_
np.array(x.columns)[coeff == 0]

array([], dtype=object)

<IPython.core.display.Javascript object>

In [21]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns

Index(['CaO', 'MgO', 'Na2O', 'Al2O3', 'SiO2', 'SO3', 'K2O', 'Fe2O3',
       'Loss on Ignition', 'Insoluble Residue', 'Total C3S', 'Alpha C2S',
       'Beta C2S', 'Gamma C2S', 'C4AF', 'C3A', 'Cubic C3A', 'Orthorhombic C3A',
       'Free CaO', 'Portlandite', 'Periclase', 'Arcanite', 'Aphthitalite',
       'Gypsum', 'Bassanite', 'Anhydrite', 'Calcite', 'Dolomite', 'Quartz',
       'std_ch_feats', 'ratio_CaO_to_SiO2', 'ratio_MgO_to_CaO',
       'mean_mi_set1_feats', 'mean_mi_set2_feats',
       'ratio_Aluminate_to_Ferrite'],
      dtype='object')

<IPython.core.display.Javascript object>

In [22]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns.shape

(35,)

<IPython.core.display.Javascript object>

In [23]:
x.columns.shape

(35,)

<IPython.core.display.Javascript object>

In [24]:
coeffs = pd.DataFrame(
    {col: [c] for col, c in zip(x.columns, coeff)}, index=["Coefficients"]
)

<IPython.core.display.Javascript object>

In [25]:
coeffs.T["Coefficients"].sort_values(ascending=False).to_frame(
    name="Coefficients"
).style.background_gradient(axis=None, vmin=1, vmax=5, cmap="Greens")

,Coefficients
CaO,169.106902
Beta C2S,44.674337
SiO2,34.205991
Alpha C2S,18.512376
Cubic C3A,14.921730
MgO,14.864808
C3A,14.412392
ratio_CaO_to_SiO2,13.903410
Gamma C2S,10.944478
Orthorhombic C3A,10.506245


<IPython.core.display.Javascript object>

In [26]:
pd.concat(results_to_save).groupby(
    ["Features", "Model", "Cross Validation", "Cross Validation Params"]
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                       
0  Chemical + Mineralogical + Feature Engineering  Linear Regression   
1  Chemical + Mineralogical + Feature Engineering  Linear Regression   
2  Chemical + Mineralogical + Feature Engineering  Linear Regression   
3  Chemical + Mineralogical + Feature Engineering  Linear Regression   

             Cross Validation  \
                                
0  Blocking Time Series Split   
1           Out of time Split   
2              Repeated KFold   
3           Time Series Split   

                            Cross Validation Params RMSE Test            \
                                                         mean       std   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}  3.753366  0.445944   
1                                {"Test Size": 0.2}  3.081432  0.000000   
2                     {"N_Splits": 5, "Repeats": 3}  3.442696  0.160652   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}  4.035213  0.581476   

   MAE Test           MAPE Test             R2 Test            
       mean       std      mean       std      mean       std  
0  2.930611  0.408041  0.069973  0.010322  0.412466  0.117655  
1  2.396342  0.000000  0.055980  0.000000  0.616404  0.000000  
2  2.631255  0.138236  0.063168  0.003160  0.530535  0.051513  
3  3.129473  0.432922  0.074318  0.009953  0.356876  0.150387

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [27]:
path = (
    "../../../../../../reports/results/local_models/partner_i-oficial/all_cements/full/"
)
filename = f"linear_regression_results_full_{index_to_save}.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

## Saving the grouped dataframe

In [28]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../../reports/results/local_models/partner_i-oficial/all_cements/grouped/"
filename = f"linear_regression_results_grouped_{index_to_save}.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

In [29]:
df_results_to_save

Category     Company  Data Shape Timesteps  \
                                                   
0  Local Model  partner_i  (1226, 35)       NaN   
1  Local Model  partner_i  (1226, 35)       NaN   
2  Local Model  partner_i  (1226, 35)       NaN   
3  Local Model  partner_i  (1226, 35)       NaN   

                                         Features              Model  \
                                                                       
0  Chemical + Mineralogical + Feature Engineering  Linear Regression   
1  Chemical + Mineralogical + Feature Engineering  Linear Regression   
2  Chemical + Mineralogical + Feature Engineering  Linear Regression   
3  Chemical + Mineralogical + Feature Engineering  Linear Regression   

             Cross Validation  \
                                
0  Blocking Time Series Split   
1           Out of time Split   
2              Repeated KFold   
3           Time Series Split   

                            Cross Validation Params RMSE Train            ...  \
                                                          mean       std  ...   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}   2.862668  0.330450  ...   
1                                {"Test Size": 0.2}   3.410104  0.000000  ...   
2                     {"N_Splits": 5, "Repeats": 3}   3.323717  0.039570  ...   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}   3.122463  0.261199  ...   

   R2 Train           RMSE Test            MAE Test           MAPE Test  \
       mean       std      mean       std      mean       std      mean   
0  0.678969  0.049177  3.753366  0.445944  2.930611  0.408041  0.069973   
1  0.545033  0.000000  3.081432  0.000000  2.396342  0.000000  0.055980   
2  0.565755  0.011030  3.442696  0.160652  2.631255  0.138236  0.063168   
3  0.599490  0.053892  4.035213  0.581476  3.129473  0.432922  0.074318   

              R2 Test            
        std      mean       std  
0  0.010322  0.412466  0.117655  
1  0.000000  0.616404  0.000000  
2  0.003160  0.530535  0.051513  
3  0.009953  0.356876  0.150387  

[4 rows x 24 columns]

<IPython.core.display.Javascript object>